# Punto 2
Un vendedor debe hacer un recorrido por todas y cada una de las capitales de los 32 estados de los Estados Unidos Mexicanos. 

Utilice colonias de hormigas y algoritmos genéticos para encontrar el orden óptimo. El costo de desplazamiento entre ciudades es la suma del valor de la hora del vendedor (es un parámetro que debe estudiarse), el costo de los peajes y el costo del combustible. Cada equipo debe definir en qué carro hace el recorrido el vendedor y de allí extraer el costo del combustible.

Adicionalmente represente con un gif animado o un video cómo se comporta la mejor solución usando un gráfico del recorrido en el mapa de México.

Primero, procederemos a cargar las librerías necesarias para abordar el problema

In [2]:
import pandas as pd
import folium
import numpy as np
from IPython.display import display
import random
from folium import plugins

Luego de esto, cargaremos algunos archivos necesarios, estos son:
1. Tiempos de desplazamiento entre estados: Estos datos serán necesarios para calcular el valor del pago al transportista, según también una constante que es el valor de la hora.
2. Coordenadas: Las coordenadas serán usadas para graficar el grafo que representa la ruta más óptima. Estas coordenadas corresponden a las capitales de cada Estado, se usan estas y no otras ciudades debido a que son precisamente las ciudades más importantes.
3. Distancias: Estas distancias se usarán para calcular el valor del consumo de combustible, según también una constante que es el costo de llenar un tanque de un vehículo que seleccionaremos. 

In [11]:
tiempos = pd.read_csv('tabla_tiempos.csv',index_col=0)
distancias = pd.read_csv('tabla_distancias.csv',index_col=0)
coordenadas = pd.read_csv('coordenadas_cap_mex.csv',header=None)
coordenadas = coordenadas.rename(columns={0:'latitude', 1:'longitude'})
nombres_estados = list(tiempos.columns)

In [ ]:
center_lat = coordenadas['latitude'].mean()
center_lon = coordenadas['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

for idx, row in coordenadas.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f'Capital: {nombres_estados[idx]}'
    ).add_to(m)

display(m)

In [8]:
def tiempo_dec(tiempo_str):
    if type(tiempo_str) == str:
        hh,mm = map(int, tiempo_str.split(':'))
        return round(hh+mm/60.0,4)
    else:
        return 0
tiempos = tiempos.fillna(0)
tiempos
tiempos_ = tiempos.copy().map(tiempo_dec)
times = tiempos_.to_numpy()


## Cálculo de variables de interés

### Valor de la hora * Tiempo de desplazamiento
Teniendo en cuenta que ya tenemos los tiempos de desplazamiento del vendedor, debemos crear la variable que multiplica esto por el valor de la hora. Para ello consultamos los salarios mínimos de 2024 diarios para cada tipo de profesión y oficio en esta página: https://www.gob.mx/cms/uploads/attachment/file/873886/Tabla_de_Salarios_M_nimos_2024.pdf, donde encontramos que el salario para un chofer de camioneta de carga en general (que es el trabajo de nuestro repartidor) es de $284.76 MXN; el número de horas de trabajo usuales en México es de 8 horas, por lo que el salario mínimo por hora para el chofer es $35.6 MXN aproximadamente. De esta manera, multiplicaremos este valor por el tiempo de desplazamiento entre estados:

In [16]:
val_hora_tiempo = times*35.6
print(val_hora_tiempo)

[[   0.      1050.79452 1041.89452 ...  439.66    1062.06548   72.98   ]
 [1030.02548    0.       921.44548 ... 1603.78    2225.59452  968.91452]
 [1069.78     913.73452    0.      ... 1418.06548 2040.47452  906.61452]
 ...
 [ 406.43452 1612.68    1431.12    ...    0.       712.59452  479.41452]
 [1017.56548 2223.81452 2042.25452 ...  704.28548    0.      1090.54548]
 [  69.42     984.93452  881.1     ...  508.48548 1130.3        0.     ]]


### Costo del combustible

En este caso, elegimos una camioneta Chevrolet N400 MAX, elegido debido a que es un vehículo con la que se suelen repartir paquetes. Esta tiene un tanque con capacidad de 45 litros para gasolina (https://www.chevrolet.cl/content/dam/chevrolet/south-america/chile/espanol/index/fichas-tecnicas/03-pdf/ficha-tecnica-n400.pdf). El rendimiento del combustimble de este vehículo es de 16.1 km/l en carretera (https://www.latercera.com/mtonline/noticia/chevrolet-n400-max/988955-2/#:~:text=Asimismo%2C%20ahora%20est%C3%A1%20ubicado%20en,1%20km%2Fl%20en%20carretera.).

El precio de la gasolina (Magna en este caso) ronda los $24 MXN por litro. Teniendo esta información en cuenta, procederemos a calcular esta variable, de esta manera:

$Costo \ combustible = \dfrac{distancia \ de \ un \ estado \ a \ otro}{eficiencia \ del \ combustible}*precio \ de \ la \ gasolina$

Donde la eficiencia del combustible y el precio de la gasolina son escalares, y se multiplican a nuestra matriz de distancias de un estado a otro.

In [17]:
distancias = distancias.fillna(0)
distancias_np = distancias.to_numpy()

In [20]:

cost_combustible = (distancias_np/16.1)*24
cost_combustible

array([[   0.        , 3206.45962733, 1996.02484472, ..., 1109.06832298,
        2657.88819876,  175.90062112],
       [3200.49689441,    0.        , 2018.38509317, ..., 4248.44720497,
        5797.26708075, 3029.06832298],
       [1996.02484472, 2018.38509317,    0.        , ..., 2859.13043478,
        4407.95031056, 1764.9689441 ],
       ...,
       [1103.10559006, 4245.46583851, 2856.14906832, ...,    0.        ,
        1665.0931677 , 1280.49689441],
       [2668.32298137, 5809.19254658, 4421.36645963, ..., 1663.60248447,
           0.        , 2844.22360248],
       [ 174.40993789, 3039.50310559, 1770.93167702, ..., 1283.47826087,
        2830.80745342,    0.        ]])

In [55]:
# Parámetros del algoritmo
n_states = 32  # Número de estados
alpha = 1      # Influencia de las feromonas
beta = 2       # Influencia de la visibilidad (tiempos de viaje)
evaporation_rate = 0.5
pheromone_constant = 100
n_ants = n_states  # Usar una hormiga por estado

random.seed(41)
np.random.seed(41)

# Inicializa la matriz de feromonas
pheromones = np.ones((n_states, n_states))  # Cantidad inicial de feromonas

# Cálculo de visibilidad (inversa del tiempo de viaje, para que tiempos menores sean más atractivos)
visibility = 1 / distances
visibility[visibility == np.inf] = 0  # Evitar divisiones por cero

# Ejecución del algoritmo
best_route = None
best_cost = float('inf')

for iteration in range(100):  # Número de iteraciones para el algoritmo
    all_routes = []
    all_costs = []
    
    # Para cada hormiga
    for ant in range(n_ants):
        # Escoge un estado inicial aleatorio para la hormiga
        current_state = random.randint(0, n_states - 1)
        visited_states = [current_state]
        cost = 0
        
        # Construcción de la ruta completa
        for step in range(n_states - 1):
            # Calcular probabilidades para el próximo estado
            probabilities = []
            for next_state in range(n_states):
                if next_state not in visited_states:
                    prob = (pheromones[current_state][next_state] ** alpha) * (visibility[current_state][next_state] ** beta)
                    probabilities.append((next_state, prob))
            
            # Seleccionar el próximo estado basado en las probabilidades
            next_state = random.choices(
                [state for state, _ in probabilities],
                weights=[prob for _, prob in probabilities]
            )[0]
            
            # Actualizar el costo y el estado actual
            cost += distances[current_state][next_state]
            current_state = next_state
            visited_states.append(current_state)
        
        # Completa el ciclo regresando al punto inicial
        cost += distances[current_state][visited_states[0]]
        all_routes.append(visited_states)
        all_costs.append(cost)
        
        # Actualización de la mejor ruta
        if cost < best_cost:
            best_cost = cost
            best_route = visited_states

    # Actualización de feromonas
    pheromones *= (1 - evaporation_rate)  # Evaporación de feromonas
    for route, cost in zip(all_routes, all_costs):
        for i in range(n_states - 1):
            pheromones[route[i]][route[i + 1]] += pheromone_constant / cost
        # Retorno al punto inicial
        pheromones[route[-1]][route[0]] += pheromone_constant / cost

best_route_names = [nombres_estados[i] for i in best_route]
print(best_route)
print("Mejor ruta en nombres de estados:", best_route_names)
print("Costo total en horas de la mejor ruta:", best_cost)

C:\Users\aoroz\AppData\Local\Temp\ipykernel_29372\4021206370.py:16: RuntimeWarning: divide by zero encountered in divide
  visibility = 1 / distances


[25, 1, 5, 7, 18, 27, 23, 13, 17, 8, 15, 21, 14, 6, 16, 11, 20, 28, 29, 26, 4, 3, 30, 22, 19, 12, 10, 0, 31, 9, 24, 2]
Mejor ruta en nombres de estados: ['Hermosillo', 'Mexicali', 'Chihuahua', 'Saltillo', 'Monterrey', 'Ciudad Victoria', 'San Luis Potosí', 'Guadalajara', 'Tepic', 'Colima', 'Morelia', 'Santiago de Querétaro', 'Toluca de Lerdo', 'CDMX', 'Cuernavaca', 'Chilpancingo de los Bravo', 'Heroica Puebla de Zaragoza', 'Tlaxcala de Xicohténcatl', 'Xalapa-Enríquez', 'Villahermosa', 'Tuxtla Gutiérrez', 'San Francisco de Campeche', 'Mérida', 'Chetumal', 'Oaxaca de Juárez', 'Pachuca de Soto', 'Guanajuato', 'Aguascalientes', 'Zacatecas', 'Victoria de Durango', 'Culiacán Rosales', 'La Paz']
Costo total en horas de la mejor ruta: 226.4333000000001


In [56]:
# Crear mapa base
center_lat = coordenadas['latitude'].mean()
center_lon = coordenadas['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Agregar nodos con colores según posición
for i, idx in enumerate(best_route):
    # Determinar color según posición
    if i == 0:
        color = 'red'
    elif i == len(best_route) - 1:
        color = 'green'
    else:
        color = 'blue'
        
    folium.CircleMarker(
        location=[coordenadas.iloc[idx]['latitude'], coordenadas.iloc[idx]['longitude']],
        radius=8,
        color=color,
        fill=True,
        popup=f'Stop {i}: Node {idx}',
        tooltip=str(i)
    ).add_to(m)

# Conectar nodos con flechas
for i in range(len(best_route)-1):
    start_idx = best_route[i]
    end_idx = best_route[i+1]
    coords = [
        [coordenadas.iloc[start_idx]['latitude'], coordenadas.iloc[start_idx]['longitude']],
        [coordenadas.iloc[end_idx]['latitude'], coordenadas.iloc[end_idx]['longitude']]
    ]
    line = folium.PolyLine(coords, weight=2, color='blue', opacity=0.8).add_to(m)
    plugins.PolyLineTextPath(line, '>', repeat=True, offset=8).add_to(m)

display(m)